# F1 Pit Stop Strategy - Model Training

**Goal:** Train an ML model to predict optimal pit stop timing

**What we're predicting:** `LapNumber` - when the pit stop occurs

**Features (inputs):** 13 variables including tire age, position, speed, track conditions

**Algorithm:** XGBoost Regressor (predicts continuous values)

**Success Metrics:**
- MAE (Mean Absolute Error) - How many laps off are predictions?
- R² Score - How well does model explain variance?
- RMSE (Root Mean Squared Error) - Penalizes large errors

**Why this matters:** Accurate pit timing can save 10-20 seconds per race!

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings("ignore")



In [2]:
df = pd.read_csv('data/processed/ml_features.csv')

print("Data loaded successfully.")
print(f" Rows: {len(df)}")
print(f" Columns: {len(df.columns)}")

print("Columns:", df.columns.tolist())

print("first 5 rows:")
print(df.head())

Data loaded successfully.
 Rows: 725
 Columns: 14
Columns: ['LapNumber', 'TyreLife', 'FreshTyre', 'Position', 'Stint', 'race_round', 'SpeedFL', 'LapTime_seconds', 'has_safety_car', 'has_vsc', 'has_yellow', 'Compound_encoded', 'Driver_encoded', 'RaceName_encoded']
first 5 rows:
   LapNumber  TyreLife  FreshTyre  Position  Stint  race_round  SpeedFL  \
0       15.0       1.0      False       2.0    2.0           1    279.0   
1       37.0       1.0       True       1.0    3.0           1    281.0   
2       10.0       1.0       True      20.0    2.0           1    278.0   
3       26.0       1.0       True      17.0    3.0           1    280.0   
4       41.0       1.0       True      10.0    4.0           1    289.0   

   LapTime_seconds  has_safety_car  has_vsc  has_yellow  Compound_encoded  \
0          118.378               0        0           0                 1   
1          118.490               0        0           0                 3   
2          121.262               0      

In [3]:
target = 'LapNumber'

feature_columns = [col for col in df.columns if col != target]

X = df[feature_columns]
y = df[target]

print(f"\n🎯 Target variable: {target}")
print(f"   Shape: {y.shape}")
print(f"   Min lap: {y.min()}")
print(f"   Max lap: {y.max()}")
print(f"   Average: {y.mean():.1f}")

print(f"\n📊 Features: {len(feature_columns)} variables")
print(f"   Shape: {X.shape}")
print(f"\n📋 Feature list:")
for i, col in enumerate(feature_columns, 1):
    print(f"   {i:2}. {col}")


🎯 Target variable: LapNumber
   Shape: (725,)
   Min lap: 1.0
   Max lap: 71.0
   Average: 26.4

📊 Features: 13 variables
   Shape: (725, 13)

📋 Feature list:
    1. TyreLife
    2. FreshTyre
    3. Position
    4. Stint
    5. race_round
    6. SpeedFL
    7. LapTime_seconds
    8. has_safety_car
    9. has_vsc
   10. has_yellow
   11. Compound_encoded
   12. Driver_encoded
   13. RaceName_encoded


In [4]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Data split complete!")
print(f"\n📊 Training set:")
print(f"   X_train shape: {X_train.shape}")
print(f"   y_train shape: {y_train.shape}")
print(f"   Pit stops for training: {len(X_train)}")

print(f"\n📊 Testing set:")
print(f"   X_test shape: {X_test.shape}")
print(f"   y_test shape: {y_test.shape}")
print(f"   Pit stops for testing: {len(X_test)}")

print(f"\n🎯 We'll train on {len(X_train)} pit stops")
print(f"🎯 We'll test on {len(X_test)} pit stops")

print(f"\n📈 Target variable distribution:")
print(f"   Training avg: {y_train.mean():.1f} laps")
print(f"   Testing avg: {y_test.mean():.1f} laps")

✅ Data split complete!

📊 Training set:
   X_train shape: (580, 13)
   y_train shape: (580,)
   Pit stops for training: 580

📊 Testing set:
   X_test shape: (145, 13)
   y_test shape: (145,)
   Pit stops for testing: 145

🎯 We'll train on 580 pit stops
🎯 We'll test on 145 pit stops

📈 Target variable distribution:
   Training avg: 26.8 laps
   Testing avg: 25.1 laps


In [5]:
#XGBoost Regressor training
model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    verbosity=0
)

print("✅ Model created!")
print(f"\n📋 Model parameters:")
print(f"   Trees: {model.n_estimators}")
print(f"   Learning rate: {model.learning_rate}")
print(f"   Max depth: {model.max_depth}")

print(f"\n🎓 Training model on {len(X_train)} pit stops...")
print("   (This might take 5-10 seconds...)")

model.fit(X_train, y_train)

print("\n🎉 MODEL TRAINED SUCCESSFULLY!")
print(f"   The model has learned patterns from {len(X_train)} pit stops!")
print("   Now it can predict pit timing for new races!")

✅ Model created!

📋 Model parameters:
   Trees: 100
   Learning rate: 0.1
   Max depth: 6

🎓 Training model on 580 pit stops...
   (This might take 5-10 seconds...)

🎉 MODEL TRAINED SUCCESSFULLY!
   The model has learned patterns from 580 pit stops!
   Now it can predict pit timing for new races!


In [6]:
print("🔮 Making predictions on test set...")

# Predict on training data (to see how well it learned)
y_train_pred = model.predict(X_train)

# Predict on test data (the real test!)
y_test_pred = model.predict(X_test)

print("✅ Predictions complete!")

print(f"\n📊 Training set predictions:")
print(f"   Made {len(y_train_pred)} predictions")
print(f"   Example predictions: {y_train_pred[:5].round(1)}")
print(f"   Actual values:       {y_train.values[:5]}")

print(f"\n📊 Test set predictions:")
print(f"   Made {len(y_test_pred)} predictions")
print(f"   Example predictions: {y_test_pred[:5].round(1)}")
print(f"   Actual values:       {y_test.values[:5]}")

# Compare a few predictions
print(f"\n🔍 Sample comparisons (first 10 from test set):")
comparison = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted': y_test_pred[:10].round(1),
    'Difference': (y_test.values[:10] - y_test_pred[:10]).round(1)
})
print(comparison)

print(f"\n💡 Difference = How many laps off the prediction was")
print(f"   Positive = Predicted too early")
print(f"   Negative = Predicted too late")

🔮 Making predictions on test set...
✅ Predictions complete!

📊 Training set predictions:
   Made 580 predictions
   Example predictions: [67.4 34.3 44.2 32.7 61.4]
   Actual values:       [68. 35. 44. 33. 61.]

📊 Test set predictions:
   Made 145 predictions
   Example predictions: [33.7 24.4 41.4 12.9 25.7]
   Actual values:       [35.  6. 40. 13. 25.]

🔍 Sample comparisons (first 10 from test set):
   Actual  Predicted  Difference
0    35.0  33.700001         1.3
1     6.0  24.400000       -18.4
2    40.0  41.400002        -1.4
3    13.0  12.900000         0.1
4    25.0  25.700001        -0.7
5     6.0  15.000000        -9.0
6    36.0  41.599998        -5.6
7     3.0   6.900000        -3.9
8    24.0  31.700001        -7.7
9    38.0  38.900002        -0.9

💡 Difference = How many laps off the prediction was
   Positive = Predicted too early
   Negative = Predicted too late


In [7]:
# Calculate metrics for training set
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# Calculate metrics for test set
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print("📊 MODEL PERFORMANCE METRICS")
print("=" * 60)

print("\n🎓 Training Set Performance:")
print(f"   MAE (Mean Absolute Error):  {train_mae:.2f} laps")
print(f"   RMSE (Root Mean Squared):    {train_rmse:.2f} laps")
print(f"   R² Score:                    {train_r2:.3f}")

print("\n🎯 Test Set Performance (THE REAL TEST):")
print(f"   MAE (Mean Absolute Error):  {test_mae:.2f} laps")
print(f"   RMSE (Root Mean Squared):    {test_rmse:.2f} laps")
print(f"   R² Score:                    {test_r2:.3f}")

print("\n" + "=" * 60)
print("📖 WHAT THESE METRICS MEAN:")
print("=" * 60)

print(f"\n🎯 MAE (Mean Absolute Error): {test_mae:.2f} laps")
print(f"   → On average, predictions are off by {test_mae:.1f} laps")
print(f"   → Lower is better (0 = perfect)")

print(f"\n🎯 RMSE: {test_rmse:.2f} laps")
print(f"   → Penalizes large errors more heavily")
print(f"   → {test_rmse:.1f} laps error including outliers")

print(f"\n🎯 R² Score: {test_r2:.3f}")
print(f"   → Model explains {test_r2*100:.1f}% of variance in pit timing")
print(f"   → 1.0 = perfect, 0.0 = random guessing")
if test_r2 > 0.7:
    print(f"   → ✅ GOOD! Model captures most patterns")
elif test_r2 > 0.5:
    print(f"   → 👍 Decent! Model has predictive power")
else:
    print(f"   → ⚠️  Room for improvement")

# Compare to baseline (always guessing average)
baseline_mae = np.mean(np.abs(y_test - y_test.mean()))
improvement = ((baseline_mae - test_mae) / baseline_mae) * 100

print(f"\n📈 Improvement over baseline:")
print(f"   Baseline (always guess {y_test.mean():.1f}): {baseline_mae:.2f} laps error")
print(f"   Our model:                                {test_mae:.2f} laps error")
print(f"   → {improvement:.1f}% better than guessing average!")

📊 MODEL PERFORMANCE METRICS

🎓 Training Set Performance:
   MAE (Mean Absolute Error):  1.26 laps
   RMSE (Root Mean Squared):    1.84 laps
   R² Score:                    0.987

🎯 Test Set Performance (THE REAL TEST):
   MAE (Mean Absolute Error):  4.47 laps
   RMSE (Root Mean Squared):    6.35 laps
   R² Score:                    0.851

📖 WHAT THESE METRICS MEAN:

🎯 MAE (Mean Absolute Error): 4.47 laps
   → On average, predictions are off by 4.5 laps
   → Lower is better (0 = perfect)

🎯 RMSE: 6.35 laps
   → Penalizes large errors more heavily
   → 6.4 laps error including outliers

🎯 R² Score: 0.851
   → Model explains 85.1% of variance in pit timing
   → 1.0 = perfect, 0.0 = random guessing
   → ✅ GOOD! Model captures most patterns

📈 Improvement over baseline:
   Baseline (always guess 25.1): 13.92 laps error
   Our model:                                4.47 laps error
   → 67.9% better than guessing average!
